# Calculate First Four Moments

In [2]:
import numpy as np
from scipy.stats import norm, kurtosis
from scipy.stats import ttest_1samp
from statistics import mean, variance
import threading

# Simulate based on the defined Distribution, N(0, 1)
n = 1000
np.random.seed(0)
sim = np.random.normal(0, 1, n)

# Function to calculate the first four moments
def first4Moments(sample):
    n = len(sample)

    # Mean
    μ_hat = sum(sample) / n

    # Remove the mean from the sample
    sim_corrected = sample - μ_hat
    cm2 = np.dot(sim_corrected, sim_corrected) / n

    # Variance
    σ2_hat = np.dot(sim_corrected, sim_corrected) / (n - 1)

    # Skewness
    skew_hat = sum(sim_corrected ** 3) / n / (cm2 ** (3 / 2))

    # Kurtosis
    kurt_hat = sum(sim_corrected ** 4) / n / (cm2 ** 2)

    excessKurt_hat = kurt_hat - 3

    return μ_hat, σ2_hat, skew_hat, excessKurt_hat

m, s2, sk, k = first4Moments(sim)

print(f"Mean {m} ({mean(sim)})")
print(f"Variance {s2} ({variance(sim)})")
print(f"Skewness {sk} ({mean(sim) - 0})")
print(f"Kurtosis {k} ({kurtosis(sim, fisher=False)})")
print(f"Mean diff = {m - mean(sim)}")
print(f"Variance diff = {s2 - variance(sim)}")
print(f"Skewness diff = {sk - (mean(sim) - 0)}")
print(f"Kurtosis diff = {k - kurtosis(sim, fisher=False)}")

# Study the limiting expected values from the estimators
sample_size = 1000
samples = 100

means = np.empty(samples)
vars = np.empty(samples)
skews = np.empty(samples)
kurts = np.empty(samples)

def calculate_moments(i):
    sample = np.random.normal(0, 1, sample_size)
    means[i], vars[i], skews[i], kurts[i] = first4Moments(sample)

threads = []
for i in range(samples):
    thread = threading.Thread(target=calculate_moments, args=(i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print(f"Mean versus Expected {mean(means) - 0}")
print(f"Variance versus Expected {mean(vars) - 1}")
print(f"Skewness versus Expected {mean(skews) - 0}")
print(f"Kurtosis versus Expected {mean(kurts) - 3}")

Mean -0.04525670749019531 (-0.045256707490195384)
Variance 0.9752096659781323 (0.9752096659781324)
Skewness 0.03385895323565697 (-0.045256707490195384)
Kurtosis -0.046766324478328514 (2.9532336755216706)
Mean diff = 7.632783294297951e-17
Variance diff = -1.1102230246251565e-16
Skewness diff = 0.07911566072585235
Kurtosis diff = -2.999999999999999
Mean versus Expected 0.0025410303279901378
Variance versus Expected -0.0049480026878993
Skewness versus Expected -0.00736035878848957
Kurtosis versus Expected -2.9757268966324713


# Hypothesis Testing

In [3]:
import numpy as np
from scipy.stats import kurtosis, t, ttest_1samp

# Set parameters
sample_size = 100000
samples = 100

# Generate samples and compute kurtosis for each sample
kurts = np.array([kurtosis(np.random.normal(0, 1, sample_size)) for _ in range(samples)])

# Summary statistics
mean_kurts = np.mean(kurts)
var_kurts = np.var(kurts, ddof=1)
std_err = np.sqrt(var_kurts / samples)

# T-statistic and p-value
t_stat = mean_kurts / std_err
p_value = 2 * (1 - t.cdf(abs(t_stat), df=samples - 1))

print(f"p-value: {p_value}")

# Using the included T-test
ttest_result = ttest_1samp(kurts, 0)  # 0 is the expected kurtosis of the normal distribution
p_value_2 = ttest_result.pvalue
print(f"p-value2: {p_value_2}")

print(f"Match the stats package test?: {np.isclose(p_value, p_value_2)}")

p-value: 0.131387823913794
p-value2: 0.13138782391379403
Match the stats package test?: True
